In [1]:
import pandas as pd
import time

time_start=time.time()

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

train=train[['qid','uid','target']]
test=test[['qid','uid']]
member_info=member_info[['uid','l_topic']]
ques_info=ques_info[['qid','qtopic']]
print(len(member_info),len(ques_info))
member_info.drop_duplicates(['uid'],inplace=True)
ques_info.drop_duplicates(['qid'],inplace=True)
print(len(member_info),len(ques_info))

1931654 1829900
1931654 1829900


In [3]:
train=pd.merge(train,member_info,on='uid',how='left')
test=pd.merge(test,member_info,on='uid',how='left')

train=pd.merge(train,ques_info,on='qid',how='left')
test=pd.merge(test,ques_info,on='qid',how='left')

In [4]:
print(len(train)+len(test))

10630845


In [5]:
target=train['target']
train.drop(['target'],axis=1,inplace=True)
all_data=pd.concat([train,test],axis=0)
all_data.shape

(10630845, 4)

In [6]:
all_data['l_topic']=all_data['l_topic'].astype(str)
all_data['qtopic']=all_data['qtopic'].astype(str)
all_data['l_topic']=all_data['l_topic'].apply(lambda x:x.split(','))
all_data['qtopic']=all_data['qtopic'].apply(lambda x:x.split(','))
all_data['num_ltopic']=all_data['l_topic'].apply(lambda x:len(x))
all_data['num_qtopic']=all_data['qtopic'].apply(lambda x:len(x))

In [7]:
all_data.head()

,qid,uid,l_topic,qtopic,num_ltopic,num_qtopic
0,Q2166419046,M401693808,"[T1727, T5310, T3402, T916, T1506, T26329, T72...","[T456, T112, T9566, T5310]",23,4
1,Q1550017551,M3392373099,"[T42595, T3, T8520, T597, T6485, T6212, T25664...","[T2, T3095]",11,2
2,Q604029601,M2317670257,"[T610, T448, T61, T2801, T9019, T65, T233, T19...","[T6090, T2156, T97, T456]",45,4
3,Q2350061229,M1618461867,"[T5, T33331, T2274, T31, T245, T516, T309, T13...",[T856],32,1
4,Q2443223942,M3544409350,[-1],"[T26, T76, T17]",1,3


In [8]:
all_data['l_topic']=all_data['l_topic'].apply(lambda x:' '.join(x))
all_data['qtopic']=all_data['qtopic'].apply(lambda x:' '.join(x))

In [9]:
from tqdm import tqdm
tqdm.pandas()

In [10]:
import numpy as np
#import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import gc
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [11]:
tfidf = CountVectorizer(ngram_range=(1,1))
tf_ltopic = tfidf.fit_transform(all_data['l_topic'])
tfidf = CountVectorizer(ngram_range=(1,1))
tf_qtopic = tfidf.fit_transform(all_data['qtopic'])

In [12]:
print(tf_ltopic.shape)
print(tf_qtopic.shape)

(10630845, 79097)
(10630845, 55246)


In [13]:
from scipy import sparse
all_sparse=sparse.csc_matrix(sparse.hstack((tf_ltopic, tf_qtopic)))
all_sparse.shape

(10630845, 134343)

In [14]:
train_feature=all_sparse[0:train.shape[0]]
test_feature=all_sparse[train.shape[0]:]

In [15]:
def lrstacking():
    num_class=2
    df_stack=pd.DataFrame()
    n_folds=5
    gc.collect()
    print('LR stacking')
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (trn_idx, val_idx) in enumerate(folds.split(target)):
        print('stack:%d' % ((i + 1) ))
        clf = LogisticRegression(random_state=1017, C=8,solver='sag',verbose=1,n_jobs=-1)
        clf.fit(train_feature[trn_idx], target[trn_idx])
        score_va = clf.predict_proba(train_feature[val_idx])
        score_te = clf.predict_proba(test_feature)
        stack_train[val_idx, :] = score_va
        stack_test[:, :] += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_lr_{}'.format(i)] = stack[:, i]
    print('lr done')
    return df_stack

In [16]:
lrstacking=lrstacking()

LR stacking
stack:1
max_iter reached after 1269 seconds


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 21.2min finished


stack:2
max_iter reached after 1348 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 22.5min finished


stack:3
max_iter reached after 1242 seconds


...

stack:4
max_iter reached after 1212 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 20.2min finished


stack:5
max_iter reached after 1183 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 19.7min finished


lr done


In [17]:
def sgdstacking():
    gc.collect()
    print('sgd stacking')
    df_stack=pd.DataFrame()
    n_folds=5
    num_class=2
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        sgd = SGDClassifier(random_state=1017,
                            verbose=1,
                            loss='log',
                            penalty='elasticnet',
                            shuffle=True,
                            n_jobs=20,
                            l1_ratio=0.0001,
                            alpha=1e-05,
                            class_weight=None)
        gc.collect()
        sgd.fit(train_feature[tr], target[tr])
        score_va = sgd.predict_proba(train_feature[va])
        score_te = sgd.predict_proba(test_feature)
        stack_train[va] = score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_sgd_{}'.format(i)] = stack[:, i]
    print('sgd done')
    return df_stack

In [18]:
sgdstacking=sgdstacking()

sgd stacking
stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


-- Epoch 1
Norm: 27.22, NNZs: 128462, Bias: -1.560385, T: 7591329, Avg. loss: 0.595555
Total training time: 7.65 seconds.
-- Epoch 2
Norm: 25.78, NNZs: 128430, Bias: -1.557176, T: 15182658, Avg. loss: 0.454066
Total training time: 14.85 seconds.
-- Epoch 3
Norm: 25.42, NNZs: 128492, Bias: -1.554335, T: 22773987, Avg. loss: 0.450993
Total training time: 22.32 seconds.
-- Epoch 4
Norm: 25.27, NNZs: 128427, Bias: -1.555018, T: 30365316, Avg. loss: 0.449744
Total training time: 29.71 seconds.
-- Epoch 5
Norm: 25.17, NNZs: 128480, Bias: -1.560710, T: 37956645, Avg. loss: 0.449062
Total training time: 36.99 seconds.
stack:2/5
-- Epoch 1
Norm: 27.25, NNZs: 128539, Bias: -1.562312, T: 7591329, Avg. loss: 0.598222
Total training time: 7.75 seconds.
-- Epoch 2
Norm: 25.75, NNZs: 128395, Bias: -1.565994, T: 15182658, Avg. loss: 0.454112
Total training time: 14.92 seconds.
-- Epoch 3
Norm: 25.41, NNZs: 128430, Bias: -1.555355, T: 22773987, Avg. loss: 0.451044
Total training time: 21.85 seconds.
--

In [19]:
def ridgestacking():
    gc.collect()
    print('Ridge stacking')
    df_stack=pd.DataFrame()
    num_class=2
    n_folds=5
    stack_train = np.zeros([train_feature.shape[0], num_class])
    stack_test = np.zeros([test_feature.shape[0], num_class])
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(target)):
        print('stack:%d/%d' % ((i + 1), n_folds))
        ridge = RidgeClassifier(random_state=1017,solver='sag',)
        ridge.fit(train_feature[tr], target[tr])
        score_va = ridge._predict_proba_lr(train_feature[va])
        score_te = ridge._predict_proba_lr(test_feature)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= n_folds
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_ridge_{}'.format(i)] = stack[:, i]
    print('ridge done')
    return df_stack

In [20]:
ridgestacking=ridgestacking()

Ridge stacking
stack:1/5
stack:2/5
stack:3/5
stack:4/5
stack:5/5
ridge done


In [21]:
def pacstacking():
    df_stack=pd.DataFrame()

    gc.collect()
    print('PAC stacking')
    stack_train = np.zeros((len(train), 2))
    stack_test = np.zeros((len(test), 2))
    score_va = 0
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(train.values, target.values)):
        print('stack:%d/%d' % ((i + 1), 5))
        pac = PassiveAggressiveClassifier(random_state=1017)
        pac.fit(train_feature[tr], target.iloc[tr])
        score_va = pac._predict_proba_lr(train_feature[va])
        score_te = pac._predict_proba_lr(test_feature)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= 5
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_pac{}'.format(i)] = stack[:, i]
    return df_stack

In [22]:
pacstacking=pacstacking()

PAC stacking
stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


stack:2/5
stack:3/5
stack:4/5
stack:5/5


In [23]:
def bnbstacking():
    df_stack=pd.DataFrame()
    gc.collect()
    print('nb stacking')
    stack_train = np.zeros((len(train), 2))
    stack_test = np.zeros((len(test), 2))
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    for i, (tr, va) in enumerate(folds.split(train.values, target.values)):
        print('stack:%d/%d' % ((i + 1), 5))
        bnb = BernoulliNB()
        bnb.fit(train_feature[tr], target.iloc[tr])
        score_va = bnb.predict_proba(train_feature[va])
        score_te = bnb.predict_proba(test_feature)
        stack_train[va] += score_va
        stack_test += score_te
    stack_test /= 5
    stack = np.vstack([stack_train, stack_test])
    for i in range(stack.shape[1]):
        df_stack['tfc_bnb{}'.format(i)] = stack[:, i]
    return df_stack

In [24]:
bnbstacking=bnbstacking()

nb stacking
stack:1/5
stack:2/5
stack:3/5
stack:4/5
stack:5/5


In [25]:
result=pd.concat([lrstacking,sgdstacking,ridgestacking,pacstacking,bnbstacking],axis=1)
result.head()

,tfc_lr_0,tfc_lr_1,tfc_sgd_0,tfc_sgd_1,tfc_ridge_0,tfc_ridge_1,tfc_pac0,tfc_pac1,tfc_bnb0,tfc_bnb1
0,0.934691,0.065309,0.916474,0.083526,0.706637,0.293363,0.755473,0.244527,0.938196,0.061804
1,0.876887,0.123113,0.833000,0.167000,0.680556,0.319444,0.280617,0.719383,0.771054,0.228946
2,0.940608,0.059392,0.936382,0.063618,0.713974,0.286026,0.898003,0.101997,0.977369,0.022631
3,0.894460,0.105540,0.864938,0.135062,0.683882,0.316118,0.982662,0.017338,0.908724,0.091276
4,0.759196,0.240804,0.770589,0.229411,0.627563,0.372437,0.543113,0.456887,0.235833,0.764167


In [26]:
result.drop(['tfc_lr_0','tfc_sgd_0','tfc_ridge_0','tfc_pac0','tfc_bnb0'],axis=1,inplace=True)

In [27]:
result.head()

,tfc_lr_1,tfc_sgd_1,tfc_ridge_1,tfc_pac1,tfc_bnb1
0,0.065309,0.083526,0.293363,0.244527,0.061804
1,0.123113,0.167000,0.319444,0.719383,0.228946
2,0.059392,0.063618,0.286026,0.101997,0.022631
3,0.105540,0.135062,0.316118,0.017338,0.091276
4,0.240804,0.229411,0.372437,0.456887,0.764167


In [28]:
result.to_csv('feature_topic_tfidf.csv',index=False)

In [29]:
all_data.head()

,qid,uid,l_topic,qtopic,num_ltopic,num_qtopic
0,Q2166419046,M401693808,T1727 T5310 T3402 T916 T1506 T26329 T7293 T180...,T456 T112 T9566 T5310,23,4
1,Q1550017551,M3392373099,T42595 T3 T8520 T597 T6485 T6212 T25664 T148 T...,T2 T3095,11,2
2,Q604029601,M2317670257,T610 T448 T61 T2801 T9019 T65 T233 T190 T55 T5...,T6090 T2156 T97 T456,45,4
3,Q2350061229,M1618461867,T5 T33331 T2274 T31 T245 T516 T309 T1326 T119 ...,T856,32,1
4,Q2443223942,M3544409350,-1,T26 T76 T17,1,3


In [30]:
len(all_data)

10630845

In [31]:
all_data[['num_ltopic','num_qtopic']].to_csv('feature_topic_count.csv',index=False)

In [32]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 19130.74426674843
